In [ ]:
!pip install umap-learn

In [44]:
import pandas as pd
import numpy as np
import warnings
import umap
warnings.filterwarnings("ignore")
from pandas.util import hash_pandas_object
from scipy.stats import spearmanr

**Подготовка данных**

In [6]:
grades = pd.read_excel("Оценки 2018-2022 ПМИ.xlsx")
grades.head()

,Номер студенческого билета,Оценка (10 баллов),Оценка (5 баллов) текстовая,Зачет/Незачет,Оценка (5 баллов),Неявка,Дата сдачи,Ведомость,Вид попытки,Считать результатом до пересдач,Образовательная программа студента
0,М161БПМИИ019,10.0,отлично,зачтено,5.0,(Нет),43362,02.26/2018.1-3,сдача,Да,Прикладная математика и информатика
1,М181БПМИИ058,7.0,хорошо,зачтено,4.0,(Нет),43362,02.26/2018.1-3,сдача,Да,Прикладная математика и информатика
2,М161БПМИИ088,9.0,отлично,зачтено,5.0,(Нет),43362,02.26/2018.1-3,сдача,Да,Прикладная математика и информатика
3,М161БПМИИ100,7.0,хорошо,зачтено,4.0,(Нет),43362,02.26/2018.1-3,сдача,Да,Прикладная математика и информатика
4,М161БПМИИ151,9.0,отлично,зачтено,5.0,(Нет),43362,02.26/2018.1-3,сдача,Да,Прикладная математика и информатика


In [7]:
grades = grades.loc[grades["Номер студенческого билета"].str.startswith('М18') == True]

In [8]:
grades = grades.drop(columns=["Оценка (5 баллов) текстовая", "Зачет/Незачет", "Оценка (5 баллов)",
                              "Неявка", "Дата сдачи", "Вид попытки", "Считать результатом до пересдач",
                              "Образовательная программа студента"])
grades = grades.rename(columns = {"Ведомость":"Номер ведомости"})
grades

,Номер студенческого билета,Оценка (10 баллов),Номер ведомости
1,М181БПМИИ058,7.0,02.26/2018.1-3
5,М181БПМИИ098,7.0,02.26/2018.1-14
8,М181БПМИИ264,10.0,02.26/2018.1-20
9,М181БПМИИ071,6.0,02.26/2018.1-19
10,М181БПМИИ303,6.0,02.26/2018.1-24
...,...,...,...
36851,М181БПМИИ227,1.0,02.26/2021.4-1074
36898,М181БПМИИ307,0.0,02.26/2021.4-1258
36901,М181БПМИИ195,4.0,02.26/2021.4-1257
36904,М181БПМИИ107,0.0,02.26/2021.4-1257


In [9]:
state = pd.read_excel("Все ведомости 2018-2022.xlsx")
state.head()

,Учебный год,Период обучения,Факультет,Номер ведомости,Дата ведомости,Вид ведомости,Запись учебного плана,Вид испытания,Кафедра,Преподаватель,...,Порядковый номер,Группа студентов,Предметная единица,Вид предметной единицы,Группа на дату формирования ведомости,Подгруппа,Языковая группа,Страновая специализация,Руководитель курсовой работы,E-mail для отправки результатов
0,"Учебный год=""2018/2019 учебный год""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018/2019 учебный год,4 модуль,отдел международной студенческой мобильности,01.91.01.02/2018.4-190,43634,сдача,Основы программирования в Python,Экзамен,департамент больших данных и информационного п...,В.М. Кохтев,...,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018/2019 учебный год,4 модуль,отдел международной студенческой мобильности,01.91.01.02/2018.4-300,43643,сдача,Анализ сетевых структур,Экзамен,департамент анализа данных и искусственного ин...,Л.Е. Жуков,...,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iyakovleva@hse.ru
3,2018/2019 учебный год,4 модуль,отдел международной студенческой мобильности,01.91.01.02/2018.4-301,43634,сдача,Современные методы принятия решений,Экзамен,департамент анализа данных и искусственного ин...,NaN,...,301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iyakovleva@hse.ru
4,2018/2019 учебный год,4 модуль,отдел международной студенческой мобильности,01.91.01.02/2018.4-302,43621,сдача,Введение в методы машинного обучения и майнинг...,Экзамен,департамент анализа данных и искусственного ин...,Д.И. Игнатов,...,302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iyakovleva@hse.ru


In [10]:
state = state.drop(columns=["Учебный год", "Дата ведомости", "Факультет", "Вид испытания", 
                            "Преподаватель", "Вид ведомости", "Кафедра", "Вид записи плана",
                            "Вид дисциплины",  "Индекс", "Языковая группа", "Страновая специализация", "Руководитель курсовой работы",
                            "Предметная единица", "Вид предметной единицы", "Подгруппа", "Порядковый номер",
                            "E-mail для отправки результатов", "Группа студентов", "Группа на дату формирования ведомости"])
state = state.iloc [1:, :]
state.head()

,Период обучения,Номер ведомости,Запись учебного плана
1,4 модуль,01.91.01.02/2018.4-190,Основы программирования в Python
2,4 модуль,01.91.01.02/2018.4-300,Анализ сетевых структур
3,4 модуль,01.91.01.02/2018.4-301,Современные методы принятия решений
4,4 модуль,01.91.01.02/2018.4-302,Введение в методы машинного обучения и майнинг...
5,4 модуль,01.91.01.02/2018.4-308,Распределенные базы и хранилища данных


In [11]:
df = pd.merge(state, grades, left_on="Номер ведомости", right_on="Номер ведомости")
df

,Период обучения,Номер ведомости,Запись учебного плана,Номер студенческого билета,Оценка (10 баллов)
0,4 модуль,02.26/2018.4-1038,Алгоритмы и структуры данных,М181БПМИИ304,3.0
1,4 модуль,02.26/2018.4-1038,Алгоритмы и структуры данных,М181БПМИИ299,3.0
2,4 модуль,02.26/2018.4-195,Практикум по математическому анализу 2,М181БПМИИ053,10.0
3,4 модуль,02.26/2018.4-195,Практикум по математическому анализу 2,М181БПМИИ153,9.0
4,4 модуль,02.26/2018.4-195,Практикум по математическому анализу 2,М181БПМИИ256,6.0
...,...,...,...,...,...
8357,2 модуль,02.26/2021.2-9,Глубинное обучение,М181БПМИИ007,0.0
8358,2 модуль,02.26/2021.2-903,Анализ данных секвенирования,М181БПМИИ107,0.0
8359,2 модуль,02.26/2021.2-904,Байесовские методы в машинном обучении,М181БПМИИ283,0.0
8360,2 модуль,02.26/2021.2-915,Глубинное обучение,М181БПМИИ307,0.0


In [12]:
module = df["Период обучения"].str.removesuffix(" модуль")

In [13]:
df["Модуль"] = module
df.head()

,Период обучения,Номер ведомости,Запись учебного плана,Номер студенческого билета,Оценка (10 баллов),Модуль
0,4 модуль,02.26/2018.4-1038,Алгоритмы и структуры данных,М181БПМИИ304,3.0,4
1,4 модуль,02.26/2018.4-1038,Алгоритмы и структуры данных,М181БПМИИ299,3.0,4
2,4 модуль,02.26/2018.4-195,Практикум по математическому анализу 2,М181БПМИИ053,10.0,4
3,4 модуль,02.26/2018.4-195,Практикум по математическому анализу 2,М181БПМИИ153,9.0,4
4,4 модуль,02.26/2018.4-195,Практикум по математическому анализу 2,М181БПМИИ256,6.0,4


In [14]:
df.isna().sum()

Период обучения                 0
Номер ведомости                 0
Запись учебного плана           0
Номер студенческого билета      0
Оценка (10 баллов)            207
Модуль                          0
dtype: int64

In [15]:
df = df.dropna()

In [16]:
df = df.drop(columns=["Период обучения", "Номер ведомости"])

In [17]:
id = hash_pandas_object(df["Номер студенческого билета"], index=False, categorize=True)
df["id"] = id
df.head()

,Запись учебного плана,Номер студенческого билета,Оценка (10 баллов),Модуль,id
0,Алгоритмы и структуры данных,М181БПМИИ304,3.0,4,8889900325791017288
1,Алгоритмы и структуры данных,М181БПМИИ299,3.0,4,13953175650191783523
2,Практикум по математическому анализу 2,М181БПМИИ053,10.0,4,4500176112568038752
3,Практикум по математическому анализу 2,М181БПМИИ153,9.0,4,11860257015020921279
4,Практикум по математическому анализу 2,М181БПМИИ256,6.0,4,12970231330059875158


In [18]:
df = df.drop(columns=["Номер студенческого билета"])
df.head()

,Запись учебного плана,Оценка (10 баллов),Модуль,id
0,Алгоритмы и структуры данных,3.0,4,8889900325791017288
1,Алгоритмы и структуры данных,3.0,4,13953175650191783523
2,Практикум по математическому анализу 2,10.0,4,4500176112568038752
3,Практикум по математическому анализу 2,9.0,4,11860257015020921279
4,Практикум по математическому анализу 2,6.0,4,12970231330059875158


In [19]:
sub_mod = []
for i in range(len(df)):
  example = df.iloc[i:(i + 1), :]
  name = str(example["Запись учебного плана"].values[0] + " " + str(example["Модуль"].values[0]))
  sub_mod.append(name)

In [20]:
df["Предмет-модуль"] = sub_mod
df.head()

,Запись учебного плана,Оценка (10 баллов),Модуль,id,Предмет-модуль
0,Алгоритмы и структуры данных,3.0,4,8889900325791017288,Алгоритмы и структуры данных 4
1,Алгоритмы и структуры данных,3.0,4,13953175650191783523,Алгоритмы и структуры данных 4
2,Практикум по математическому анализу 2,10.0,4,4500176112568038752,Практикум по математическому анализу 2 4
3,Практикум по математическому анализу 2,9.0,4,11860257015020921279,Практикум по математическому анализу 2 4
4,Практикум по математическому анализу 2,6.0,4,12970231330059875158,Практикум по математическому анализу 2 4


In [21]:
len(set(sub_mod))

150

In [22]:
sub_mod_dict = {}
code_sub = [i for i in range (0,150)]
for i in range(len(df)):
  example = df.iloc[i:(i+1), :]
  name = example["Предмет-модуль"].values[0]
  if name in sub_mod_dict:
    continue
  else:
    sub_mod_dict[name] = code_sub[0]
    code_sub.pop(0)

In [23]:
codes = []
for i in range(len(df)):
  example = df.iloc[i:(i+1), :]
  name = example["Предмет-модуль"].values[0]
  codes.append(sub_mod_dict[name])

In [24]:
df["Код предмета"] = codes
df

,Запись учебного плана,Оценка (10 баллов),Модуль,id,Предмет-модуль,Код предмета
0,Алгоритмы и структуры данных,3.0,4,8889900325791017288,Алгоритмы и структуры данных 4,0
1,Алгоритмы и структуры данных,3.0,4,13953175650191783523,Алгоритмы и структуры данных 4,0
2,Практикум по математическому анализу 2,10.0,4,4500176112568038752,Практикум по математическому анализу 2 4,1
3,Практикум по математическому анализу 2,9.0,4,11860257015020921279,Практикум по математическому анализу 2 4,1
4,Практикум по математическому анализу 2,6.0,4,12970231330059875158,Практикум по математическому анализу 2 4,1
...,...,...,...,...,...,...
8357,Глубинное обучение,0.0,2,14714980224934043140,Глубинное обучение 2,133
8358,Анализ данных секвенирования,0.0,2,7353746595378349276,Анализ данных секвенирования 2,100
8359,Байесовские методы в машинном обучении,0.0,2,16206220508047667344,Байесовские методы в машинном обучении 2,138
8360,Глубинное обучение,0.0,2,17947246888966067913,Глубинное обучение 2,133


In [25]:
df = df.drop(columns=["Модуль"])

In [26]:
code_courses = list(pd.unique(df["Код предмета"]))
code_courses.insert(0, "id")
ids = list(pd.unique(df["id"]))

In [27]:
df1 = df.groupby([df["id"], df["Код предмета"], df["Запись учебного плана"]])

In [28]:
df_dict = {k:-np.ones(len(ids)) for k in code_courses}

In [29]:
id_prev = 0
curr_id = -1
for (id, code, course), grade in df1.groups.items():
  if id != id_prev:
    curr_id += 1
    df_dict["id"][curr_id] = id
    id_prev = id
  grade = df[(df["id"] == id) & (df["Запись учебного плана"] == course)]["Оценка (10 баллов)"].values[0]
  df_dict[code][curr_id] = grade

In [30]:
data = pd.DataFrame(df_dict)
data = data.drop("id", axis=1)
data = data.dropna()
data = data.T
data

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258
0,8.0,4.0,7.0,8.0,-1.0,4.0,5.0,9.0,6.0,7.0,...,-1.0,9.0,9.0,6.0,7.0,9.0,6.0,8.0,8.0,7.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,10.0,10.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6.0,-1.0,5.0,-1.0,-1.0,-1.0,-1.0,9.0,10.0,7.0,...,-1.0,4.0,-1.0,7.0,5.0,10.0,-1.0,-1.0,-1.0,-1.0
146,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,5.0,-1.0,-1.0,-1.0,-1.0,-1.0
147,5.0,-1.0,-1.0,-1.0,-1.0,5.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0
148,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [31]:
data.to_csv("Закодированные предметы.csv", index=True)

**Алгоритм нахождения схожести предметов**

In [32]:
grades = pd.read_csv("Закодированные предметы.csv")
grades.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,249,250,251,252,253,254,255,256,257,258
0,0,8.0,4.0,7.0,8.0,-1.0,4.0,5.0,9.0,6.0,...,-1.0,9.0,9.0,6.0,7.0,9.0,6.0,8.0,8.0,7.0
1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,10.0,10.0,-1.0,-1.0
2,2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [33]:
grades = grades.rename(columns={"Unnamed: 0":"Код предмета"})
grades.head()

,Код предмета,0,1,2,3,4,5,6,7,8,...,249,250,251,252,253,254,255,256,257,258
0,0,8.0,4.0,7.0,8.0,-1.0,4.0,5.0,9.0,6.0,...,-1.0,9.0,9.0,6.0,7.0,9.0,6.0,8.0,8.0,7.0
1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,10.0,10.0,-1.0,-1.0
2,2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


*Словарь пар предметов*

In [34]:
grades_dict = {}
for i in range(len(grades) - 1):
  str_1 = grades.iloc[i:(i + 1), :]
  for j in range((i + 1), len(grades)):
    str_2 = grades.iloc[j:(j + 1), :]
    subjects = [str_1["Код предмета"].values[0], str_2["Код предмета"].values[0]]
    subjects= tuple(subjects)
    arr_1 = str_1.iloc[:, 1:].values
    arr_2 = str_2.iloc[:, 1:].values
    arr_1 = arr_1.reshape(-1, 1)
    arr_2 = arr_2.reshape(-1, 1)
    st_1_gr = []
    st_2_gr = []
    for k in range(len(arr_1)):
      if (arr_1[k] >= 0).all() and  (arr_2[k] >= 0).all(): 
        st_1_gr.append(arr_1[k])
        st_2_gr.append(arr_2[k])
    if len(st_1_gr) > 10:
      grades_dict.update({subjects:[st_1_gr , st_2_gr]})

In [35]:
print(len(grades_dict))

2346


*Словарь метрик*

In [36]:
metric_dict= {}
for key in grades_dict:
  metrics = []
  st_grades = grades_dict[key]
  l_1 = np.linalg.norm(np.array(st_grades[0]) - np.array(st_grades[1]), ord=1)
  metrics.append(l_1)
  l_2 = np.linalg.norm(np.array(st_grades[0]) - np.array(st_grades[1]), ord=2)
  metrics.append(l_2)
  corr = np.corrcoef(np.array(st_grades[0]), np.array(st_grades[1]), rowvar=False)[0,1]
  metrics.append(corr)
  rho, p = spearmanr(np.array(st_grades[0]), np.array(st_grades[1]))
  metrics.append(rho)
  metric_dict.update({key:metrics})

In [37]:
df = pd.DataFrame.from_dict(metric_dict, orient='index').reset_index()
df

,index,0,1,2,3
0,"(0, 1)",62.0,16.613248,0.145044,0.062420
1,"(0, 8)",307.0,27.110883,0.609348,0.613947
2,"(0, 11)",287.0,25.903668,0.625440,0.632571
3,"(0, 12)",306.0,26.907248,0.611115,0.606674
4,"(0, 13)",354.0,33.436507,0.456441,0.488701
...,...,...,...,...,...
2341,"(133, 140)",59.0,14.525839,0.623863,0.613194
2342,"(137, 144)",45.0,10.816654,0.414661,0.460789
2343,"(138, 140)",26.0,13.564660,0.615028,0.753401
2344,"(139, 140)",14.0,6.480741,0.815995,0.476785


In [38]:
df = df.rename(columns={"index":"Код предметов", 0:"L1", 1:"L2", 2:"Корреляция Пирсона", 3:" Корреляция Спирмена"})
df.head()

,Код предметов,L1,L2,Корреляция Пирсона,Корреляция Спирмена
0,"(0, 1)",62.0,16.613248,0.145044,0.062420
1,"(0, 8)",307.0,27.110883,0.609348,0.613947
2,"(0, 11)",287.0,25.903668,0.625440,0.632571
3,"(0, 12)",306.0,26.907248,0.611115,0.606674
4,"(0, 13)",354.0,33.436507,0.456441,0.488701


*Нахождение "Наивной схожести"*

In [39]:
sub_mod_reverse = {sub_mod_dict[k]: k for k in sub_mod_dict.keys()}

In [40]:
similarity = []
for i in range(len(df)):
   pair_sub = df.iloc[i:(i + 1), :]
   sub_1, sub_2 = pair_sub["Код предметов"].values[0]
   sub_1 = sub_mod_reverse[sub_1]
   sub_2 = sub_mod_reverse[sub_2]
   if sub_1.split()[0] == sub_2.split()[0]:
    similarity.append(1)
   else:
    similarity.append(0)

In [41]:
df["Наивная схожесть"] = similarity
df.head()

,Код предметов,L1,L2,Корреляция Пирсона,Корреляция Спирмена,Наивная схожесть
0,"(0, 1)",62.0,16.613248,0.145044,0.062420,0
1,"(0, 8)",307.0,27.110883,0.609348,0.613947,0
2,"(0, 11)",287.0,25.903668,0.625440,0.632571,0
3,"(0, 12)",306.0,26.907248,0.611115,0.606674,0
4,"(0, 13)",354.0,33.436507,0.456441,0.488701,0


*Поиск пар предметов класса 0*

In [54]:
grades_umap = grades.iloc[:, 1:].values

In [61]:
embedding = umap.UMAP(metric='correlation').fit_transform(grades_umap)
embedding.shape

(150, 2)

In [65]:
umap_dict = {}
for i in range(len(embedding)):
  str_1 = grades.iloc[i:(i + 1), :]
  for j in range((i + 1), len(embedding)):
    str_2 = grades.iloc[j:(j + 1), :]
    subjects = [str_1["Код предмета"].values[0], str_2["Код предмета"].values[0]]
    subjects= tuple(subjects)
    distance =np.linalg.norm((embedding[i] - embedding[j]), ord=2)
    arr_1 = str_1.iloc[:, 1:].values
    arr_2 = str_2.iloc[:, 1:].values
    arr_1 = arr_1.reshape(-1, 1)
    arr_2 = arr_2.reshape(-1, 1)
    st_1_gr = []
    st_2_gr = []
    for k in range(len(arr_1)):
      if (arr_1[k] >= 0).all() and  (arr_2[k] >= 0).all(): 
        st_1_gr.append(arr_1[k])
        st_2_gr.append(arr_2[k])
    if len(st_1_gr) > 10:
      umap_dict.update({subjects:distance})
len(umap_dict)

2346

In [72]:
sorted_umap_dict = {}
sorted_keys = sorted(umap_dict, key=umap_dict.get, reverse=True)
for i in sorted_keys:
    sorted_umap_dict[i] = umap_dict[i]

In [74]:
keys = list(sorted_umap_dict.keys())
keys = keys[:56]

*Обучение модели классификации*

In [83]:
train_1 = df[df["Наивная схожесть"] == 1]
train_2 = df.loc[df["Код предметов"].isin(keys)]
train = pd.concat([train_1, train_2])
train

,Код предметов,L1,L2,Корреляция Пирсона,Корреляция Спирмена,Наивная схожесть
12,"(0, 24)",0.0,0.000000,1.000000,1.000000,1
30,"(0, 62)",251.0,27.037012,0.636990,0.662260,1
31,"(0, 63)",52.0,9.899495,0.644375,0.636597,1
67,"(0, 113)",27.0,7.549834,0.686941,0.590878,1
110,"(1, 32)",14.0,5.099020,0.619841,0.588494,1
...,...,...,...,...,...,...
2296,"(116, 128)",17.0,6.708204,0.740863,0.730845,0
2297,"(116, 133)",9.0,4.358899,0.858751,0.860666,0
2298,"(116, 140)",36.0,10.392305,0.615626,0.690079,0
2316,"(121, 140)",24.0,8.246211,0.337484,0.325650,0


In [80]:
X_train = train.iloc[:, 1:5]
y_train = train.iloc[:, 5:]

In [131]:
target = df[df["Наивная схожесть"] != 1]
target = target[~df["Код предметов"].isin(keys)]
X_target = target.iloc[:, 1:5]
X_target

,L1,L2,Корреляция Пирсона,Корреляция Спирмена
0,62.0,16.613248,0.145044,0.062420
1,307.0,27.110883,0.609348,0.613947
2,287.0,25.903668,0.625440,0.632571
3,306.0,26.907248,0.611115,0.606674
4,354.0,33.436507,0.456441,0.488701
...,...,...,...,...
2338,27.0,9.746794,0.186876,0.126147
2339,22.0,8.485281,0.709493,0.794820
2342,45.0,10.816654,0.414661,0.460789
2343,26.0,13.564660,0.615028,0.753401


In [97]:
X_target.isna().sum()

L1                      0
L2                      0
Корреляция Пирсона      3
 Корреляция Спирмена    3
dtype: int64

In [98]:
X_target = X_target.dropna()

In [99]:
from sklearn.linear_model import LogisticRegression

In [100]:
clf = LogisticRegression(random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_target)

In [107]:
prob = clf.predict_proba(X_target)
len(prob)

2231

In [110]:
probabilities = []
for i in range(len(prob)):
  probabilities.append(prob[i][0])

In [132]:
target = target.dropna()
target = target.drop(columns=["Наивная схожесть"])
target["Вероятность схожести"] = probabilities
target.head()

,Код предметов,L1,L2,Корреляция Пирсона,Корреляция Спирмена,Вероятность схожести
0,"(0, 1)",62.0,16.613248,0.145044,0.062420,8.084422e-01
1,"(0, 8)",307.0,27.110883,0.609348,0.613947,3.377577e-08
2,"(0, 11)",287.0,25.903668,0.625440,0.632571,1.241266e-07
3,"(0, 12)",306.0,26.907248,0.611115,0.606674,3.407417e-08
4,"(0, 13)",354.0,33.436507,0.456441,0.488701,7.970415e-09


In [117]:
distance = -np.log(probabilities)
distance

array([ 0.2126461 , 17.20352225, 15.90196366, ...,  0.645024  ,
        0.06258859,  0.43351726])

In [133]:
target["Расстояние между предметами"] = distance
target.head()

,Код предметов,L1,L2,Корреляция Пирсона,Корреляция Спирмена,Вероятность схожести,Расстояние между предметами
0,"(0, 1)",62.0,16.613248,0.145044,0.062420,8.084422e-01,0.212646
1,"(0, 8)",307.0,27.110883,0.609348,0.613947,3.377577e-08,17.203522
2,"(0, 11)",287.0,25.903668,0.625440,0.632571,1.241266e-07,15.901964
3,"(0, 12)",306.0,26.907248,0.611115,0.606674,3.407417e-08,17.194726
4,"(0, 13)",354.0,33.436507,0.456441,0.488701,7.970415e-09,18.647529


In [123]:
pair_sub = []
for i in range(len(target)):
  str_1 = target.iloc[i:(i + 1), :]
  sub_1, sub_2 = str_1["Код предметов"].values[0]
  name_1 = sub_mod_reverse[sub_1]
  name_2 = sub_mod_reverse[sub_2]
  names = [name_1, name_2]
  names = tuple(names)
  pair_sub.append(names)
len(pair_sub)

2231

In [134]:
target["Пара предметов"] = pair_sub
target.tail()

,Код предметов,L1,L2,Корреляция Пирсона,Корреляция Спирмена,Вероятность схожести,Расстояние между предметами,Пара предметов
2338,"(131, 146)",27.0,9.746794,0.186876,0.126147,0.836888,0.178065,"(Научно-исследовательский семинар ""Математичес..."
2339,"(133, 138)",22.0,8.485281,0.709493,0.794820,0.699391,0.357545,"(Глубинное обучение 2, Байесовские методы в ма..."
2342,"(137, 144)",45.0,10.816654,0.414661,0.460789,0.524650,0.645024,"(Автоматическая обработка текста 2, Прикладная..."
2343,"(138, 140)",26.0,13.564660,0.615028,0.753401,0.939330,0.062589,"(Байесовские методы в машинном обучении 2, Глу..."
2345,"(141, 145)",36.0,10.295630,0.539434,0.515690,0.648225,0.433517,"(Дизайн систем 2, Теория отказоустойчивых расп..."


In [135]:
target = target.drop(columns=["Код предметов"])
target.tail()

,L1,L2,Корреляция Пирсона,Корреляция Спирмена,Вероятность схожести,Расстояние между предметами,Пара предметов
2338,27.0,9.746794,0.186876,0.126147,0.836888,0.178065,"(Научно-исследовательский семинар ""Математичес..."
2339,22.0,8.485281,0.709493,0.794820,0.699391,0.357545,"(Глубинное обучение 2, Байесовские методы в ма..."
2342,45.0,10.816654,0.414661,0.460789,0.524650,0.645024,"(Автоматическая обработка текста 2, Прикладная..."
2343,26.0,13.564660,0.615028,0.753401,0.939330,0.062589,"(Байесовские методы в машинном обучении 2, Глу..."
2345,36.0,10.295630,0.539434,0.515690,0.648225,0.433517,"(Дизайн систем 2, Теория отказоустойчивых расп..."


**Результаты работы алгорима**

In [142]:
maximum = target.max()
maximum

L1                                                                         625.0
L2                                                                     52.659282
Корреляция Пирсона                                                      0.944058
 Корреляция Спирмена                                                    0.947057
Вероятность схожести                                                    0.951874
Расстояние между предметами                                            34.945041
Пара предметов                 (Язык SQL 2, Глубинное обучение для текстовых ...
dtype: object

In [155]:
target[target["Расстояние между предметами"] > 34.9]

,L1,L2,Корреляция Пирсона,Корреляция Спирмена,Вероятность схожести,Расстояние между предметами,Пара предметов
403,625.0,52.659282,0.331431,0.208964,6.661338e-16,34.945041,(Архитектура компьютера и операционные системы...


In [176]:
target.tail()

,L1,L2,Корреляция Пирсона,Корреляция Спирмена,Вероятность схожести,Расстояние между предметами,Пара предметов
2338,27.0,9.746794,0.186876,0.126147,0.836888,0.178065,"(Научно-исследовательский семинар ""Математичес..."
2339,22.0,8.485281,0.709493,0.794820,0.699391,0.357545,"(Глубинное обучение 2, Байесовские методы в ма..."
2342,45.0,10.816654,0.414661,0.460789,0.524650,0.645024,"(Автоматическая обработка текста 2, Прикладная..."
2343,26.0,13.564660,0.615028,0.753401,0.939330,0.062589,"(Байесовские методы в машинном обучении 2, Глу..."
2345,36.0,10.295630,0.539434,0.515690,0.648225,0.433517,"(Дизайн систем 2, Теория отказоустойчивых расп..."
